In [1]:
## Step 1 - Import the Libraries

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import ta  # Technical Analysis library
from datetime import datetime, timedelta


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
## Step 2 – Define the training time frame
start_date = "2014-01-01"
end_date = "2024-01-01"

## Step 3 - Helper Functions
def get_dow30_tickers():
    url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average#Components'
    tables = pd.read_html(url)
    dow30_table = tables[1]
    tickers = dow30_table['Symbol'].tolist()
    return tickers

def load_stock_data(ticker, start_date, end_date):
    ticker = ticker.replace('.', '-')
    data = yf.download(ticker, start=start_date, end=end_date)
    data['returns'] = data['Adj Close'].pct_change().fillna(0)
    data['direction'] = np.where(data['Adj Close'].shift(-1) > data['Adj Close'], 1, 0)
    return data

def feature_engineering(data):
    print(f"Initial data shape: {data.shape}")

    # Applying MACD
    data['macd'] = ta.trend.macd(data['Adj Close']).fillna(0)

    # Applying Bollinger Bands
    bollinger = ta.volatility.BollingerBands(data['Adj Close'])
    data['boll_ub'] = bollinger.bollinger_hband().fillna(data['Adj Close'])
    data['boll_lb'] = bollinger.bollinger_lband().fillna(data['Adj Close'])

    # Applying RSI
    data['rsi_30'] = ta.momentum.RSIIndicator(data['Adj Close'], window=30).rsi().fillna(50)

    # Applying ADX
    if len(data) >= 30:
        adx = ta.trend.ADXIndicator(data['High'], data['Low'], data['Adj Close'], window=30)
        data['dx_30'] = adx.adx().fillna(20)
    else:
        data['dx_30'] = np.nan

    # SMA
    data['close_30_sma'] = data['Adj Close'].rolling(window=30).mean().ffill()
    data['close_60_sma'] = data['Adj Close'].rolling(window=60).mean().bfill()

    # Dropping NaN values
    data.dropna(inplace=True)
    print(f"After dropping NaN values: {data.shape}")
    
    return data



def prepare_data_for_modeling(data, features):
    output_var = data[['direction']]

    scaler = MinMaxScaler()
    feature_transform = scaler.fit_transform(data[features])
    feature_transform = pd.DataFrame(feature_transform, columns=features, index=data.index)

    output_var = output_var.values.ravel()

    train_size = int(len(feature_transform) * 0.8)
    X_train, X_test = feature_transform[:train_size], feature_transform[train_size:]
    y_train, y_test = output_var[:train_size], output_var[train_size:]

    X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

    return X_train, X_test, y_train, y_test, scaler

In [5]:
## Step 4 - Loop through Each Ticker
dow30_tickers = get_dow30_tickers()
models = {}
scalers = {}

features = ['Adj Close', 'Volume', 'boll_ub', 'boll_lb', 'rsi_30', 'close_30_sma', 'close_60_sma']

for ticker in dow30_tickers:
    print(f"Processing {ticker}...")
    try:
        data = load_stock_data(ticker, start_date, end_date)
        data = feature_engineering(data)
       
        X_train, X_test, y_train, y_test, scaler = prepare_data_for_modeling(data, features)

        ## Build and Train the LSTM Model
        lstm = Sequential()
        lstm.add(LSTM(32, input_shape=(1, X_train.shape[2]), activation='relu', return_sequences=False))
        lstm.add(Dense(1, activation='sigmoid'))
        lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        lstm.fit(X_train, y_train, epochs=100, batch_size=8, verbose=1, shuffle=False, validation_data=(X_test, y_test), callbacks=[early_stopping])

        ## Make Predictions
        y_pred = (lstm.predict(X_test) > 0.5).astype(int)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        print(f'{ticker} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

        models[ticker] = lstm
        scalers[ticker] = scaler

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

print("Finished processing all tickers.")


[*********************100%%**********************]  1 of 1 completed

Processing MMM...
Initial data shape: (2516, 8)
After MACD:                 macd
Date                
2023-12-22  2.389954
2023-12-26  2.450287
2023-12-27  2.510682
2023-12-28  2.591288
2023-12-29  2.600801
After Bollinger Bands:               boll_ub    boll_lb
Date                            
2023-12-22  88.913152  79.275502
2023-12-26  89.163122  79.927264
2023-12-27  89.539265  80.386694
2023-12-28  89.924442  80.917952
2023-12-29  90.143257  81.536343
After RSI:                rsi_30
Date                 
2023-12-22  62.468146
2023-12-26  64.418466
2023-12-27  65.082806
2023-12-28  66.071003
2023-12-29  65.326090
After ADX:                 dx_30
Date                 
2023-12-22  18.019013
2023-12-26  18.226646
2023-12-27  18.471283
2023-12-28  18.798848
2023-12-29  19.115495
After SMA:             close_30_sma  close_60_sma
Date                                  
2023-12-22     81.731357     77.002458
2023-12-26     82.187066     77.219457
2023-12-27     82.663144     77.490600
202


c:\Users\leand\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


249/249 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5462 - loss: 0.6918 - val_accuracy: 0.4659 - val_loss: 0.6968
Epoch 2/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5462 - loss: 0.6902 - val_accuracy: 0.4659 - val_loss: 0.6973
Epoch 3/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5462 - loss: 0.6900 - val_accuracy: 0.4659 - val_loss: 0.6979
Epoch 4/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5462 - loss: 0.6898 - val_accuracy: 0.4659 - val_loss: 0.6983
Epoch 5/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5462 - loss: 0.6896 - val_accuracy: 0.4659 - val_loss: 0.6987
Epoch 6/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5462 - loss: 0.6895 - val_accuracy: 0.4659 - val_loss: 0.6990
Epoch 7/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5462 - loss: 0.6894 - val_accuracy: 0.4659 - val_loss: 0.6994
Epoch 8/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5462 - loss: 0.6893 - val_accuracy: 0.4659

[*********************100%%**********************]  1 of 1 completed
c:\Users\leand\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

249/249 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5161 - loss: 0.6914 - val_accuracy: 0.5181 - val_loss: 0.6941
Epoch 2/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5327 - loss: 0.6911 - val_accuracy: 0.5181 - val_loss: 0.6938
Epoch 3/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5327 - loss: 0.6908 - val_accuracy: 0.5181 - val_loss: 0.6937
Epoch 4/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5327 - loss: 0.6908 - val_accuracy: 0.5181 - val_loss: 0.6935
Epoch 5/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5327 - loss: 0.6908 - val_accuracy: 0.5181 - val_loss: 0.6934
Epoch 6/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5327 - loss: 0.6908 - val_accuracy: 0.5181 - val_loss: 0.6932
Epoch 7/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5327 - loss: 0.6909 - val_accuracy: 0.5181 - val_loss: 0.6931
Epoch 8/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5327 - loss: 0.6909 - val_accuracy: 0.5181

[*********************100%%**********************]  1 of 1 completed

Initial data shape: (2516, 8)
After MACD:                 macd
Date                
2023-12-22 -0.319304
2023-12-26 -0.243216
2023-12-27 -0.142009
2023-12-28 -0.069285
2023-12-29 -0.009405
After Bollinger Bands:               boll_ub    boll_lb
Date                            
2023-12-22  52.238773  49.776829
2023-12-26  52.251080  49.771808
2023-12-27  52.236280  49.780882
2023-12-28  52.356325  49.729283
2023-12-29  52.485821  49.683747
After RSI:                rsi_30
Date                 
2023-12-22  49.341277
2023-12-26  49.037735
2023-12-27  50.949645
2023-12-28  50.536043
2023-12-29  50.642997
After ADX:                 dx_30
Date                 
2023-12-22  13.616418
2023-12-26  13.276956
2023-12-27  12.843049
2023-12-28  12.447747
2023-12-29  12.038234
After SMA:             close_30_sma  close_60_sma
Date                                  
2023-12-22     51.646541     52.429363
2023-12-26     51.545403     52.413206
2023-12-27     51.444105     52.404226
2023-12-28     51.339


c:\Users\leand\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


249/249 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5114 - loss: 0.6930 - val_accuracy: 0.5442 - val_loss: 0.6907
Epoch 2/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5145 - loss: 0.6928 - val_accuracy: 0.5442 - val_loss: 0.6911
Epoch 3/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5145 - loss: 0.6928 - val_accuracy: 0.5442 - val_loss: 0.6915
Epoch 4/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5145 - loss: 0.6928 - val_accuracy: 0.5442 - val_loss: 0.6916
Epoch 5/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5145 - loss: 0.6929 - val_accuracy: 0.5442 - val_loss: 0.6919
Epoch 6/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5145 - loss: 0.6929 - val_accuracy: 0.5442 - val_loss: 0.6920
Epoch 7/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5145 - loss: 0.6930 - val_accuracy: 0.5502 - val_loss: 0.6922
Epoch 8/100
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5145 - loss: 0.6930 - val_accuracy: 0.5341

In [6]:
## Step 5 - Generate Future Predictions
def generate_predictions(models, scalers, tickers, historical_data, future_end_date):
    predictions = {}
    for ticker in tickers:
        model = models.get(ticker)
        scaler = scalers.get(ticker)
        if model and scaler:
            try:
                # Load historical data up to the end of the current week
                historical_data_ticker = historical_data[ticker]
                new_data = load_stock_data(ticker, "2024-01-01", future_end_date)
                
                # Append the new data to the historical data
                combined_data = pd.concat([historical_data_ticker, new_data])
                
                # Apply feature engineering to the combined dataset
                combined_data = feature_engineering(combined_data)
                
                # Ensure we have enough data after feature engineering
                if len(combined_data) < 60:
                    print(f"Insufficient data for {ticker} after feature engineering. Skipping.")
                    continue
                
                future_features = combined_data[features]
                
                # Transform features using the scaler
                future_features_scaled = scaler.transform(future_features)
                future_features_scaled = np.array(future_features_scaled).reshape((future_features_scaled.shape[0], 1, future_features_scaled.shape[1]))
                
                # Predict probabilities
                predictions[ticker] = model.predict(future_features_scaled).flatten()
                
            except Exception as e:
                print(f"Error processing future data for {ticker}: {e}")
    return predictions


In [7]:

## Step 6 - Generate Trading Signals
def generate_trading_signals(predictions):
    signals = {}
    for ticker, pred_probs in predictions.items():
        # Compute average confidence score
        avg_confidence = np.mean(pred_probs)
        signals[ticker] = avg_confidence
    return signals

## Step 7 - Portfolio Allocation
def allocate_portfolio(signals, total_investment):
    allocations = {}
    total_confidence = sum(signals.values())
   
    if total_confidence > 0:
        for ticker, confidence in signals.items():
            # Allocate proportionally based on confidence
            allocations[ticker] = (confidence / total_confidence) * total_investment
    return allocations



In [8]:
## Step 8 - Portfolio Construction
def initialize_portfolio(initial_date, models, scalers, tickers, start_date, end_date, initial_investment):
    historical_data = {}
    for ticker in tickers:
        try:
            data = load_stock_data(ticker, start_date, end_date)
            historical_data[ticker] = feature_engineering(data)
        except Exception as e:
            print(f"Error loading data for {ticker}: {e}")
    
    predictions = generate_predictions(models, scalers, tickers, historical_data, initial_date)
    signals = generate_trading_signals(predictions)
    allocations = allocate_portfolio(signals, initial_investment)
    return allocations, predictions


# Rebalance Portfolio Weekly
def rebalance_portfolio(start_date, end_date, initial_portfolio, initial_predictions, models, scalers, tickers, historical_data):
    current_portfolio = initial_portfolio.copy()
    portfolio_history = []
    weekly_returns = []
    
    current_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    while current_date <= end_date:
        week_end_date = current_date + timedelta(days=7)
        
        predictions = generate_predictions(models, scalers, tickers, historical_data, week_end_date.strftime("%Y-%m-%d"))
        signals = generate_trading_signals(predictions)
        
        # Calculate portfolio returns based on new predictions
        current_returns = 0
        for ticker in current_portfolio.keys():
            if ticker in signals:
                new_signal = signals[ticker]
                old_signal = initial_predictions[ticker]
                
                # Sell if direction changes or buy more if confidence increases
                if new_signal < 0.5 and old_signal >= 0.5:
                    current_returns += current_portfolio[ticker]  # Sell
                    current_portfolio[ticker] = 0
                elif new_signal > old_signal:
                    additional_investment = current_returns * 0.1  # Reinvest 10% of returns
                    current_portfolio[ticker] += additional_investment
                    current_returns -= additional_investment
        
        weekly_returns.append(current_returns)
        portfolio_history.append(current_portfolio.copy())
        
        current_date = week_end_date
    
    return portfolio_history, weekly_returns

initial_investment = 1000000  # Example investment of $1,000,000
simulation_start_date = "2024-01-07"
simulation_end_date = "2024-08-01"

initial_portfolio, initial_predictions = initialize_portfolio(
    simulation_start_date,
    models,
    scalers,
    dow30_tickers,
    start_date="2014-01-01",
    end_date="2024-01-01",
    initial_investment=initial_investment
)

# Run Rebalancing
portfolio_history, weekly_returns = rebalance_portfolio(simulation_start_date, simulation_end_date, initial_portfolio, initial_predictions, models, scalers, dow30_tickers, load_stock_data)

# Print Results
print("Portfolio History:")
for i, week in enumerate(portfolio_history):
    print(f"Week {i+1}: {week}")

print("Weekly Returns:")
for i, ret in enumerate(weekly_returns):
    print(f"Week {i+1}: ${ret:.2f}")

[*********************100%%**********************]  1 of 1 completed

Initial data shape: (2516, 8)
After MACD:                 macd
Date                
2023-12-22  2.389954
2023-12-26  2.450287
2023-12-27  2.510682
2023-12-28  2.591288
2023-12-29  2.600801
After Bollinger Bands:               boll_ub    boll_lb
Date                            
2023-12-22  88.913152  79.275502
2023-12-26  89.163122  79.927264
2023-12-27  89.539265  80.386694
2023-12-28  89.924442  80.917952
2023-12-29  90.143257  81.536343
After RSI:                rsi_30
Date                 
2023-12-22  62.468146
2023-12-26  64.418466
2023-12-27  65.082806
2023-12-28  66.071003
2023-12-29  65.326090



[*********************100%%**********************]  1 of 1 completed


After ADX:                 dx_30
Date                 
2023-12-22  18.019013
2023-12-26  18.226646
2023-12-27  18.471283
2023-12-28  18.798848
2023-12-29  19.115495
After SMA:             close_30_sma  close_60_sma
Date                                  
2023-12-22     81.731357     77.002458
2023-12-26     82.187066     77.219457
2023-12-27     82.663144     77.490600
2023-12-28     83.103974     77.793840
2023-12-29     83.485141     78.102363
After dropping NaN values: (2487, 15)
Initial data shape: (2516, 8)
After MACD:                 macd
Date                
2023-12-22  6.506604
2023-12-26  6.583286
2023-12-27  6.655277
2023-12-28  6.694323
2023-12-29  6.612270
After Bollinger Bands:                boll_ub     boll_lb
Date                              
2023-12-22  186.207809  158.873356
2023-12-26  187.387964  159.864926
2023-12-27  188.623254  160.749842
2023-12-28  189.856706  161.534548
2023-12-29  190.974155  162.058772
After RSI:                rsi_30
Date                 
2

[*********************100%%**********************]  1 of 1 completed

...

Initial data shape: (2491, 15)
After MACD:                 macd
Date                
2023-12-29  2.695728
2024-01-02  2.499744
2024-01-03  2.247116
2024-01-04  2.152864
2024-01-05  2.060835
After Bollinger Bands:                boll_ub     boll_lb
Date                              
2023-12-29  260.815841  252.129180
2024-01-02  260.898197  252.287443
2024-01-03  260.745954  252.791668
2024-01-04  260.669120  253.365648
2024-01-05  260.423145  254.148547
After RSI:                rsi_30
Date                 
2023-12-29  61.435291
2024-01-02  59.439884
2024-01-03  58.262701
2024-01-04  59.772111
2024-01-05  59.845884
After ADX:                 dx_30
Date                 
2023-12-29  18.153372
2024-01-02  18.222687
2024-01-03  18.202188
2024-01-04  18.339751
2024-01-05  18.501961
After SMA:             close_30_sma  close_60_sma
Date                                  
2023-12-29    254.713095    245.670439
2024-01-02    255.054471    246.099414
2024-01-03    255.333538    246.487843
2024-0

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


[*********************100%%**********************]  1 of 1 completed

Initial data shape: (2491, 15)
After MACD:                 macd
Date                
2023-12-29 -0.009405
2024-01-02  0.080928
2024-01-03  0.151043
2024-01-04  0.163585
2024-01-05  0.143818
After Bollinger Bands:               boll_ub    boll_lb
Date                            
2023-12-29  52.485821  49.683747
2024-01-02  52.746854  49.605493
2024-01-03  52.972604  49.564173
2024-01-04  53.066236  49.556823
2024-01-05  53.120494  49.609826
After RSI:                rsi_30
Date                 
2023-12-29  50.642997
2024-01-02  52.807254
2024-01-03  52.820291
2024-01-04  50.591648
2024-01-05  49.129567
After ADX:                 dx_30
Date                 
2023-12-29  12.038234
2024-01-02  11.791828
2024-01-03  11.633732
2024-01-04  11.312952
2024-01-05  11.073096
After SMA:             close_30_sma  close_60_sma
Date                                  
2023-12-29     51.212792     52.383890
2024-01-02     51.255028     52.388297
2024-01-03     51.304961     52.407436
2024-01-04     51.33

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Error processing future data for MMM: 'function' object is not subscriptable
Error processing future data for AXP: 'function' object is not subscriptable
Error processing future data for AMGN: 'function' object is not subscriptable
Error processing future data for AMZN: 'function' object is not subscriptable
Error processing future data for AAPL: 'function' object is not subscriptable
Error processing future data for BA: 'function' object is not subscriptable
Error processing future data for CAT: 'function' object is not subscriptable
Error processing future data for CVX: 'function' object is not subscriptable
Error processing future data for CSCO: 'function' object is not subscriptable
Error processing future data for KO: 'function' object is not subscriptable
Error processing future data for DIS: 'function' object is not subscriptable
Error processing future data for DOW: 'function' object is not subscriptable
Error processing future data for GS